In [7]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import img_to_array, load_img, save_img, ImageDataGenerator

# === Paths ===
RAW_DIR = "/teamspace/studios/this_studio/car_model_detection/data/raw"
AUG_DIR = "/teamspace/studios/this_studio/car_model_detection/data/augmented"
TARGET_SIZE = (224, 224)
AUG_PER_IMAGE = 5  # Number of Keras augmentations per image

# === Custom augmentation: grayscale and blur ===
def custom_augmentations(img):
    aug_images = []

    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    aug_images.append(gray_rgb)

    # Blur
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    aug_images.append(blur)

    return aug_images

# === Keras ImageDataGenerator for live augmentation ===
keras_aug = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# === Loop through each class ===
for class_name in os.listdir(RAW_DIR):
    class_dir = os.path.join(RAW_DIR, class_name)
    save_class_dir = os.path.join(AUG_DIR, 'train', class_name)
    os.makedirs(save_class_dir, exist_ok=True)

    if os.path.isdir(class_dir):
        for img_file in tqdm(os.listdir(class_dir), desc=f"Augmenting {class_name}"):
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                # Load and resize image
                img_path = os.path.join(class_dir, img_file)
                img = load_img(img_path, target_size=TARGET_SIZE)
                img_array = img_to_array(img)
                img_expanded = np.expand_dims(img_array, axis=0)

                # Save original resized image
                save_img(os.path.join(save_class_dir, f"{os.path.splitext(img_file)[0]}_orig.jpg"), img_array)

                # Live augmentations via Keras
                aug_iter = keras_aug.flow(img_expanded, batch_size=1)
                for i in range(AUG_PER_IMAGE):
                    aug_img = next(aug_iter)[0].astype(np.uint8)
                    save_img(os.path.join(save_class_dir, f"{os.path.splitext(img_file)[0]}_keras_{i}.jpg"), aug_img)

                # Custom grayscale and blur
                img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                custom_augs = custom_augmentations(img_cv)
                for i, aug_img in enumerate(custom_augs):
                    save_img(os.path.join(save_class_dir, f"{os.path.splitext(img_file)[0]}_custom_{i}.jpg"), aug_img)

print("✅ Augmentation complete. Images saved in: /data/augmented/train/<class_name>")

Augmenting toyota_corolla: 100%|██████████| 300/300 [00:20<00:00, 14.93it/s]

✅ Augmentation complete. Images saved in: /data/augmented/train/<class_name>
